In [1]:
import warnings
warnings.filterwarnings('ignore')

# 5章 顧客の退会を予測する１０本ノック

引き続き、スポーツジムの会員データを使って顧客の行動を分析していきます。  
３章では顧客の全体像を把握し、4章では数ヶ月利用している顧客の来月の利用回数の予測を行いました。   
ここでは、教師あり学習の分類を用いて、顧客の退会予測を取り扱います。

### ノック41：データを読み込んで利用データを整形しよう

In [2]:
import pandas as pd
customer = pd.read_csv('customer_join.csv')
uselog_months = pd.read_csv('use_log_months.csv')

FileNotFoundError: ignored

In [ ]:
year_months = list(uselog_months["年月"].unique())
uselog = pd.DataFrame()
for i in range(1, len(year_months)):
    tmp = uselog_months.loc[uselog_months["年月"]==year_months[i]]
    tmp.rename(columns={"count":"count_0"}, inplace=True)
    tmp_before = uselog_months.loc[uselog_months["年月"]==year_months[i-1]]
    del tmp_before["年月"]
    tmp_before.rename(columns={"count":"count_1"}, inplace=True)
    tmp = pd.merge(tmp, tmp_before, on="customer_id", how="left")
    uselog = pd.concat([uselog, tmp], ignore_index=True)
uselog.head()

### ノック42：退会前月の退会顧客データを作成しよう

In [ ]:
from dateutil.relativedelta import relativedelta
exit_customer = customer.loc[customer["is_deleted"]==1]
exit_customer["exit_date"] = None
exit_customer["end_date"] = pd.to_datetime(exit_customer["end_date"])
for i in range(len(exit_customer)):
    exit_customer["exit_date"].iloc[i] = exit_customer["end_date"].iloc[i] - relativedelta(months=1)
exit_customer["年月"] = exit_customer["exit_date"].dt.strftime("%Y%m")
uselog["年月"] = uselog["年月"].astype(str)
exit_uselog = pd.merge(uselog, exit_customer, on=["customer_id", "年月"], how="left")
print(len(uselog))
exit_uselog.head()

33851


,年月,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
0,201805,AS002855,5,4.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,201805,AS009373,4,3.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,201805,AS015233,7,NaN,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,201805,AS015315,3,6.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,201805,AS015739,5,7.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
exit_uselog = exit_uselog.dropna(subset=["name"])
print(len(exit_uselog))
print(len(exit_uselog["customer_id"].unique()))
exit_uselog.head()

1104
1104


,年月,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
19,201805,AS055680,3,3.0,XXXXX,C01,M,2018-03-01,2018-06-30,CA1,...,10500.0,通常,3.000000,3.0,3.0,3.0,0.0,2018-06-30,3.0,2018-05-30 00:00:00
57,201805,AS169823,2,3.0,XX,C01,M,2017-11-01,2018-06-30,CA1,...,10500.0,通常,3.000000,3.0,4.0,2.0,1.0,2018-06-30,7.0,2018-05-30 00:00:00
110,201805,AS305860,5,3.0,XXXX,C01,M,2017-06-01,2018-06-30,CA1,...,10500.0,通常,3.333333,3.0,5.0,2.0,0.0,2018-06-30,12.0,2018-05-30 00:00:00
128,201805,AS363699,5,3.0,XXXXX,C01,M,2018-02-01,2018-06-30,CA1,...,10500.0,通常,3.333333,3.0,5.0,2.0,0.0,2018-06-30,4.0,2018-05-30 00:00:00
147,201805,AS417696,1,4.0,XX,C03,F,2017-09-01,2018-06-30,CA1,...,6000.0,通常,2.000000,1.0,4.0,1.0,0.0,2018-06-30,9.0,2018-05-30 00:00:00


### ノック43：継続顧客のデータを作成しよう

In [ ]:
conti_customer = customer.loc[customer["is_deleted"]==0]
conti_uselog = pd.merge(uselog, conti_customer, on=["customer_id"], how="left")
print(len(conti_uselog))
conti_uselog = conti_uselog.dropna(subset=["name"])
print(len(conti_uselog))

33851
27422


In [ ]:
conti_uselog = conti_uselog.sample(frac=1).reset_index(drop=True)
conti_uselog = conti_uselog.drop_duplicates(subset="customer_id")
print(len(conti_uselog))
conti_uselog.head()

2842


,年月,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,class_name,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period
0,201902,GD257285,5,5.0,XXXX,C01,M,2017-07-01,NaN,CA2,...,オールタイム,10500.0,入会費半額,5.583333,6.0,7.0,4.0,1.0,2019-04-30,21.0
1,201806,PL558375,7,5.0,XXXX,C02,F,2017-11-01,NaN,CA1,...,デイタイム,7500.0,通常,6.250000,6.0,9.0,5.0,1.0,2019-04-30,17.0
2,201903,TS885968,4,4.0,XXXXX,C01,F,2017-06-01,NaN,CA1,...,オールタイム,10500.0,通常,6.083333,6.0,10.0,3.0,1.0,2019-04-30,22.0
3,201806,HD476406,7,6.0,XXXXX,C01,F,2016-06-01,NaN,CA1,...,オールタイム,10500.0,通常,5.083333,5.5,8.0,2.0,1.0,2019-04-30,34.0
4,201806,HI708658,7,4.0,XXX,C01,F,2015-08-01,NaN,CA1,...,オールタイム,10500.0,通常,4.166667,4.0,7.0,1.0,1.0,2019-04-30,44.0


In [ ]:
predict_data = pd.concat([conti_uselog, exit_uselog],ignore_index=True)
print(len(predict_data))
predict_data.head()

3946


,calc_date,campaign_id,campaign_name,class,class_name,count_0,count_1,customer_id,end_date,exit_date,...,max,mean,median,membership_period,min,name,price,routine_flg,start_date,年月
0,2019-04-30,CA2,入会費半額,C01,オールタイム,5,5.0,GD257285,NaN,NaN,...,7.0,5.583333,6.0,21.0,4.0,XXXX,10500.0,1.0,2017-07-01,201902
1,2019-04-30,CA1,通常,C02,デイタイム,7,5.0,PL558375,NaN,NaN,...,9.0,6.250000,6.0,17.0,5.0,XXXX,7500.0,1.0,2017-11-01,201806
2,2019-04-30,CA1,通常,C01,オールタイム,4,4.0,TS885968,NaN,NaN,...,10.0,6.083333,6.0,22.0,3.0,XXXXX,10500.0,1.0,2017-06-01,201903
3,2019-04-30,CA1,通常,C01,オールタイム,7,6.0,HD476406,NaN,NaN,...,8.0,5.083333,5.5,34.0,2.0,XXXXX,10500.0,1.0,2016-06-01,201806
4,2019-04-30,CA1,通常,C01,オールタイム,7,4.0,HI708658,NaN,NaN,...,7.0,4.166667,4.0,44.0,1.0,XXX,10500.0,1.0,2015-08-01,201806


### ノック44：予測する月の在籍期間を作成しよう

In [ ]:
predict_data["period"] = 0
predict_data["now_date"] = pd.to_datetime(predict_data["年月"], format="%Y%m")
predict_data["start_date"] = pd.to_datetime(predict_data["start_date"])
for i in range(len(predict_data)):
    delta = relativedelta(predict_data["now_date"][i], predict_data["start_date"][i])
    predict_data["period"][i] = int(delta.years*12 + delta.months)
predict_data.head()

,calc_date,campaign_id,campaign_name,class,class_name,count_0,count_1,customer_id,end_date,exit_date,...,median,membership_period,min,name,price,routine_flg,start_date,年月,period,now_date
0,2019-04-30,CA2,入会費半額,C01,オールタイム,5,5.0,GD257285,NaN,NaN,...,6.0,21.0,4.0,XXXX,10500.0,1.0,2017-07-01,201902,19,2019-02-01
1,2019-04-30,CA1,通常,C02,デイタイム,7,5.0,PL558375,NaN,NaN,...,6.0,17.0,5.0,XXXX,7500.0,1.0,2017-11-01,201806,7,2018-06-01
2,2019-04-30,CA1,通常,C01,オールタイム,4,4.0,TS885968,NaN,NaN,...,6.0,22.0,3.0,XXXXX,10500.0,1.0,2017-06-01,201903,21,2019-03-01
3,2019-04-30,CA1,通常,C01,オールタイム,7,6.0,HD476406,NaN,NaN,...,5.5,34.0,2.0,XXXXX,10500.0,1.0,2016-06-01,201806,24,2018-06-01
4,2019-04-30,CA1,通常,C01,オールタイム,7,4.0,HI708658,NaN,NaN,...,4.0,44.0,1.0,XXX,10500.0,1.0,2015-08-01,201806,34,2018-06-01


### ノック45：欠損値を除去しよう

In [ ]:
predict_data.isna().sum()

calc_date               0
campaign_id             0
campaign_name           0
class                   0
class_name              0
count_0                 0
count_1               259
customer_id             0
end_date             2842
exit_date            2842
gender                  0
is_deleted              0
max                     0
mean                    0
median                  0
membership_period       0
min                     0
name                    0
price                   0
routine_flg             0
start_date              0
年月                      0
period                  0
now_date                0
dtype: int64

In [ ]:
predict_data = predict_data.dropna(subset=["count_1"])
predict_data.isna().sum()

calc_date               0
campaign_id             0
campaign_name           0
class                   0
class_name              0
count_0                 0
count_1                 0
customer_id             0
end_date             2635
exit_date            2635
gender                  0
is_deleted              0
max                     0
mean                    0
median                  0
membership_period       0
min                     0
name                    0
price                   0
routine_flg             0
start_date              0
年月                      0
period                  0
now_date                0
dtype: int64

### ノック46：文字列型の変数を処理できるように整形しよう

In [ ]:
target_col = ["campaign_name", "class_name", "gender", "count_1", "routine_flg", "period", "is_deleted"]
predict_data = predict_data[target_col]
predict_data.head()

,campaign_name,class_name,gender,count_1,routine_flg,period,is_deleted
0,入会費半額,オールタイム,M,5.0,1.0,19,0.0
1,通常,デイタイム,F,5.0,1.0,7,0.0
2,通常,オールタイム,F,4.0,1.0,21,0.0
3,通常,オールタイム,F,6.0,1.0,24,0.0
4,通常,オールタイム,F,4.0,1.0,34,0.0


In [ ]:
predict_data = pd.get_dummies(predict_data)
predict_data.head()

,count_1,routine_flg,period,is_deleted,campaign_name_入会費半額,campaign_name_入会費無料,campaign_name_通常,class_name_オールタイム,class_name_デイタイム,class_name_ナイト,gender_F,gender_M
0,5.0,1.0,19,0.0,1,0,0,1,0,0,0,1
1,5.0,1.0,7,0.0,0,0,1,0,1,0,1,0
2,4.0,1.0,21,0.0,0,0,1,1,0,0,1,0
3,6.0,1.0,24,0.0,0,0,1,1,0,0,1,0
4,4.0,1.0,34,0.0,0,0,1,1,0,0,1,0


In [ ]:
del predict_data["campaign_name_通常"]
del predict_data["class_name_ナイト"]
del predict_data["gender_M"]
predict_data.head()

,count_1,routine_flg,period,is_deleted,campaign_name_入会費半額,campaign_name_入会費無料,class_name_オールタイム,class_name_デイタイム,gender_F
0,5.0,1.0,19,0.0,1,0,1,0,0
1,5.0,1.0,7,0.0,0,0,0,1,1
2,4.0,1.0,21,0.0,0,0,1,0,1
3,6.0,1.0,24,0.0,0,0,1,0,1
4,4.0,1.0,34,0.0,0,0,1,0,1


### ノック47：決定木を用いて退会予測モデルを作成してみよう

In [ ]:
from sklearn.tree import DecisionTreeClassifier
import sklearn.model_selection

exit = predict_data.loc[predict_data["is_deleted"]==1]
conti = predict_data.loc[predict_data["is_deleted"]==0].sample(len(exit))

X = pd.concat([exit, conti], ignore_index=True)
y = X["is_deleted"]
del X["is_deleted"]
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X,y)

model = DecisionTreeClassifier(random_state=0)
model.fit(X_train, y_train)
y_test_pred = model.predict(X_test)
print(y_test_pred)

[1. 1. 0. 0. 0. 0. 0. 0. 1. 0. 1. 1. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 1. 0.
 0. 0. 0. 1. 1. 0. 1. 0. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 0. 0. 1. 0. 0.
 1. 1. 0. 0. 1. 0. 1. 1. 0. 0. 0. 1. 1. 1. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0.
 1. 0. 0. 0. 1. 1. 0. 1. 0. 1. 0. 1. 1. 1. 1. 0. 0. 1. 1. 1. 1. 1. 0. 1.
 0. 0. 0. 0. 1. 0. 1. 1. 0. 0. 0. 0. 1. 1. 0. 1. 1. 0. 1. 1. 0. 0. 1. 0.
 0. 0. 0. 0. 1. 1. 1. 0. 1. 0. 1. 0. 0. 1. 0. 1. 1. 1. 0. 1. 0. 1. 1. 1.
 1. 0. 0. 0. 0. 1. 0. 0. 0. 1. 1. 0. 1. 1. 1. 1. 0. 0. 0. 1. 1. 0. 0. 1.
 1. 1. 0. 0. 1. 1. 0. 0. 0. 1. 0. 0. 0. 1. 1. 0. 1. 1. 0. 0. 1. 1. 1. 0.
 0. 0. 1. 0. 0. 0. 0. 1. 1. 0. 0. 1. 0. 1. 1. 1. 0. 0. 0. 0. 0. 1. 1. 0.
 1. 0. 0. 0. 1. 1. 1. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0.
 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 1. 1. 1. 0. 1. 0. 1. 0. 0. 1. 1.
 0. 1. 0. 1. 1. 1. 0. 1. 0. 1. 0. 1. 1. 0. 1. 0. 0. 0. 1. 0. 1. 0. 0. 0.
 1. 1. 0. 0. 1. 1. 0. 0. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0.
 1. 1. 0. 1. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 1. 0.

In [ ]:
results_test = pd.DataFrame({"y_test":y_test ,"y_pred":y_test_pred })
results_test.head()

,y_test,y_pred
1308,0.0,1.0
111,1.0,1.0
1645,0.0,0.0
1746,0.0,0.0
1948,0.0,0.0


### ノック48：予測モデルの評価を行ない、モデルのチューニングをしてみよう

In [ ]:
correct = len(results_test.loc[results_test["y_test"]==results_test["y_pred"]])
data_count = len(results_test)
score_test = correct / data_count
print(score_test)

0.8802281368821293


In [ ]:
print(model.score(X_test, y_test))
print(model.score(X_train, y_train))

0.8802281368821293
0.9759188846641318


In [ ]:
X = pd.concat([exit, conti], ignore_index=True)
y = X["is_deleted"]
del X["is_deleted"]
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X,y)

model = DecisionTreeClassifier(random_state=0, max_depth=5)
model.fit(X_train, y_train)
print(model.score(X_test, y_test))
print(model.score(X_train, y_train))

0.9182509505703422
0.9214195183776933


### ノック49：モデルに寄与している変数を確認しよう

In [ ]:
importance = pd.DataFrame({"feature_names":X.columns, "coefficient":model.feature_importances_})
importance

,feature_names,coefficient
0,count_1,0.334445
1,routine_flg,0.138804
2,period,0.524600
3,campaign_name_入会費半額,0.000000
4,campaign_name_入会費無料,0.000000
5,class_name_オールタイム,0.000000
6,class_name_デイタイム,0.002151
7,gender_F,0.000000


### ノック50：顧客の退会を予測しよう

In [ ]:
count_1 = 3
routine_flg = 1
period = 10
campaign_name = "入会費無料"
class_name = "オールタイム"
gender = "M"

In [ ]:
if campaign_name == "入会費半額":
    campaign_name_list = [1, 0]
elif campaign_name == "入会費無料":
    campaign_name_list = [0, 1]
elif campaign_name == "通常":
    campaign_name_list = [0, 0]
if class_name == "オールタイム":
    class_name_list = [1, 0]
elif class_name == "デイタイム":
    class_name_list = [0, 1]
elif class_name == "ナイト":
    class_name_list = [0, 0]
if gender == "F":
    gender_list = [1]
elif gender == "M":
    gender_list = [0]
input_data = [count_1, routine_flg, period]
input_data.extend(campaign_name_list)
input_data.extend(class_name_list)
input_data.extend(gender_list)

In [ ]:
print(model.predict([input_data]))
print(model.predict_proba([input_data]))

[1.]
[[0.00278552 0.99721448]]
